<a href="https://colab.research.google.com/github/dpshepherd/GeoMXprocessing/blob/main/Process_GeoMX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install dependencies

In [5]:
!pip install ome-types tifffile zarr pathlib numpy scikit-image tqdm

  Using cached zarr-2.16.1-py3-none-any.whl (206 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.9 MB/s eta 0:00:00
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5034 sha256=c5555e3bcaa0a5372f224ee9dfea1bfc326aa5cccebe18afec85a1d9381ba2b9
  Stored in directory: /root/.cache/pip/wheels/7f/4e/be/1171b40f43b918087657ec57cf3b81fa1a2e027d8755baa184
Successfully built asciitree


### Import dependencies

In [8]:
from ome_types import from_tiff
from tifffile import TiffWriter, imread
import zarr
from pathlib import Path
import numpy as np
from skimage.draw import polygon2mask
import argparse
import sys
from tqdm import tqdm
from google.colab import drive

### Connect to Google Drive

In [9]:
drive.mount('/content/drive')

### Select file using Colab interface

In [ ]:
data_path = r''

### Setup saving directories

In [ ]:
save_path = data_path.parent / Path('Extracted_ROIs')
save_path.mkdir(parents=True, exist_ok=True)
ROI_maximal_save_dir = save_path / Path('ROI_maximal')
ROI_maximal_save_dir.mkdir(parents=True, exist_ok=True)
ROI_masks_save_dir = save_path / Path('binary_masks')
ROI_masks_save_dir.mkdir(parents=True, exist_ok=True)
ROI_masked_save_dir = save_path / Path('ROI_masked')
ROI_masked_save_dir.mkdir(parents=True, exist_ok=True)

### Extract and write ROIs

In [ ]:
resolution = 0
scale = 1

# load OME-XML and image data into memory
ome_xml = from_tiff(data_path,validate=True)
store = imread(data_path,aszarr=True)
z = zarr.open(store,mode='r')
data = z[resolution]

# load XY pixel size
pixel_size_x = float(np.round(ome_xml.images[0].pixels.physical_size_x,3)) * scale
pixel_size_y = float(np.round(ome_xml.images[0].pixels.physical_size_y,3)) * scale

# load channel names
channel_names = []
for ch_idx in range(0,len(ome_xml.images[0].pixels.channels)):
    channel_names.append(ome_xml.images[0].pixels.channels[ch_idx].name)

# loop over all ROIs
for idx in tqdm(range(0,len(ome_xml.rois)),desc='ROI'):

    # extract ROI xy location in overview image
    x = int(ome_xml.rois[idx].union[2].x) // scale
    y = int(ome_xml.rois[idx].union[2].y) // scale

    # extract ROI maximal bounding box size
    height = int(ome_xml.rois[idx].union[2].height) // scale
    width = int(ome_xml.rois[idx].union[2].width) // scale

    # create ROI name
    ROI_idx = int(ome_xml.rois[idx].id.split(':')[1])
    ROI_name = 'ROI-'+str(ROI_idx+1).zfill(3)+'_maximal'
    ROI_save_path = ROI_maximal_save_dir / Path(ROI_name+'.ome.tif')

    # extract ROI data from overview image
    maximal_ROI_data = np.array(data[:,y:y+height,x:x+width])

    # write ROI to OME-TIFF format
    with TiffWriter(ROI_save_path,bigtiff=False) as tif:
        metadata = {'Name' : ROI_name,
                    'axes' : 'CYX',
                    'PhysicalSizeX' : pixel_size_x,
                    'PhysicalSizeXUnit': 'µm',
                    'PhysicalSizeY' : pixel_size_y,
                    'PhysicalSizeYUnit' : 'µm',
                    'Channel': {'Name' : channel_names}
                    }
        tif.write(data=maximal_ROI_data,
                  metadata=metadata,
                  resolution=(1e4/pixel_size_x,1e4/pixel_size_y),
                  resolutionunit='CENTIMETER',
                  photometric='minisblack')

    del ROI_name, ROI_save_path

    # attempt to extract ROI mask as polygon
    create_masked_tiff = True
    try:
        polygon_string = ome_xml.rois[idx].union[1].points
    except:
        create_masked_tiff = False

    # if polygon is found, create masked ROI image
    if create_masked_tiff:

        # convert polygon from string to (x,y) points
        polygon_pts = np.round(np.array(polygon_string.replace(' ',',').split(','),dtype=float).reshape(-1,2),0).astype(float)

        # switch polygon to (y,x) order
        polygon_pts[:,[1,0]] = polygon_pts[:,[0,1]]
        polygon_pts = polygon_pts / scale

        # translate polygon points from overview image to local ROI coordinates
        polygon_pts[:,0] = polygon_pts[:,0] - y
        polygon_pts[:,1] = polygon_pts[:,1] - x

        # create mask from polygon points
        mask = polygon2mask((height,width),polygon_pts).astype(int)

        # apply mask to ROI
        maximal_ROI_data[:,mask == 0] = 0

        ROI_name = 'ROI-'+str(ROI_idx+1).zfill(3)+'_binarymask'
        ROI_save_path = ROI_masks_save_dir / Path(ROI_name+'.ome.tif')

        # write ROI to OME-TIFF format
        with TiffWriter(ROI_save_path,bigtiff=False) as tif:
            metadata = {'Name' : ROI_name,
                        'axes' : 'YX',
                        'PhysicalSizeX' : pixel_size_x,
                        'PhysicalSizeXUnit': 'µm',
                        'PhysicalSizeY' : pixel_size_y,
                        'PhysicalSizeYUnit' : 'µm',
                        }
            tif.write(data=mask.astype(np.uint8),
                    metadata=metadata,
                    resolution=(1e4/pixel_size_x,1e4/pixel_size_y),
                    resolutionunit='CENTIMETER',
                    photometric='minisblack')

        # create masked ROI name
        ROI_name = 'ROI-'+str(ROI_idx+1).zfill(3)+'_masked'
        ROI_save_path = ROI_masked_save_dir / Path(ROI_name+'.ome.tif')

        # write ROI to OME-TIFF format
        with TiffWriter(ROI_save_path,bigtiff=False) as tif:
            metadata = {'Name' : ROI_name,
                        'axes' : 'CYX',
                        'PhysicalSizeX' : pixel_size_x,
                        'PhysicalSizeXUnit': 'µm',
                        'PhysicalSizeY' : pixel_size_y,
                        'PhysicalSizeYUnit' : 'µm',
                        'Channel': {'Name' : channel_names}
                        }
            tif.write(data=maximal_ROI_data.astype(np.uint16),
                    metadata=metadata,
                    resolution=(1e4/pixel_size_x,1e4/pixel_size_y),
                    resolutionunit='CENTIMETER',
                    photometric='minisblack')
        del mask

    # if no mask, assume full ROI was used and rewrite
    else:
        # create masked ROI name
        ROI_name = 'ROI-'+str(ROI_idx+1).zfill(3)+'_masked'
        ROI_save_path = ROI_masked_save_dir / Path(ROI_name+'.ome.tif')

        # write ROI to OME-TIFF format
        with TiffWriter(ROI_save_path,bigtiff=False) as tif:
            metadata = {'Name' : ROI_name,
                        'axes' : 'CYX',
                        'PhysicalSizeX' : pixel_size_x,
                        'PhysicalSizeXUnit': 'µm',
                        'PhysicalSizeY' : pixel_size_y,
                        'PhysicalSizeYUnit' : 'µm',
                        'Channel': {'Name' : channel_names}
                        }
            tif.write(data=maximal_ROI_data,
                    metadata=metadata,
                    resolution=(1e4/pixel_size_x,1e4/pixel_size_y),
                    resolutionunit='CENTIMETER',
                    photometric='minisblack')
    del maximal_ROI_data

drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')